In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

from matplotlib import pyplot as plt

pd.set_option('display.max_rows', None)

In [ ]:
#Open csv file.

data = pd.read_csv("/content/drive/MyDrive/NSQIP-PCF/pcf_combined.csv", index_col=0, na_values = -99)
data.head()

In [ ]:
#See all columns.

print(list(data.columns))

In [ ]:
#Check data shape.

data.shape

In [ ]:
#Define variables of interest (predictor variables, inclusion/exclusion criteria, outcomes of interest).

variables = ['SEX', 'RACE_NEW', 'ETHNICITY_HISPANIC', 'CPT', 'INOUT', 'TRANST', 'AGE', 'DISCHDEST', 'ANESTHES', 'SURGSPEC', 'ELECTSURG', 'HEIGHT', 'WEIGHT', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'DISCANCR', 'WNDINF', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'PRSEPIS', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRSGOT', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT1', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10', 'EMERGNCY', 'WNDCLAS', 'ASACLAS', 'OPTIME', 'TOTHLOS', 'HTOODAY', 'NSUPINFEC', 'NWNDINFD', 'NORGSPCSSI', 'NDEHIS', 'NOUPNEUMO', 'NREINTUB', 'NPULEMBOL', 'NFAILWEAN', 'NRENAINSF', 'NOPRENAFL', 'NURNINFEC', 'NCNSCVA', 'NCDARREST', 'NCDMI', 'NOTHBLEED', 'NOTHDVT', 'NOTHSYSEP', 'NOTHSESHOCK', 'PODIAG', 'PODIAG10', 'STILLINHOSP', 'READMISSION1']

In [ ]:
#Remove unwanted columns and check data shape.

data = data[variables]

data.shape

In [ ]:
#See the ICD codes' unique value counts for the patient cohort.

icd_codes_df = data['PODIAG10'].value_counts(normalize=False, dropna=False).to_frame()
icd_codes_index = icd_codes_df.index.tolist()

In [ ]:
#Exclude patients with ICD codes that were used less than 10 in the patient population.

icd_codes_df = data['PODIAG10'].value_counts(normalize=False, dropna=False).to_frame()

icd_codes_df.columns =['Value']

icd_codes_df = icd_codes_df[icd_codes_df['Value'] >= 10]

icd_to_include = icd_codes_df.index.tolist()

data = data[data.PODIAG10.isin(icd_to_include)]

In [ ]:
#Get the descriptions for ICD codes.

icd10 = pd.read_csv("/content/drive/MyDrive/NSQIP-PCF/icd10_descriptions.csv", index_col = 'PODIAG10', encoding = 'latin1', low_memory = False)
icd10 = icd10.filter(items = icd_to_include, axis=0)

In [ ]:
#Save the ICD codes with descriptions and value counts.

icd = pd.concat([icd10, icd_codes_df], axis=1)
icd.to_csv('/content/drive/MyDrive/NSQIP-PCF/icd.csv')
icd

In [ ]:
#Review the ICD table for excluding patients with exclude patients diagnosed with a fracture, neoplasm, infection, instrumentation related complications or lumbar/thoracic/sacral site diagnoses.

icd_to_exclude = ['C79.49', 'C79.51', 'G06.1', 'M46.22', 'M84.58XA', 'S12.000A', 'S12.100A', 'S12.300A', 'S12.400A', 'S12.500A', 'S12.590A', 'S12.9XXA']

data = data[~data.PODIAG10.isin(icd_to_exclude)]

In [ ]:
#Drop patients with missing ICD codes.

data = data[data['PODIAG10'].notna()]

In [ ]:
#Check data for ICD codes.

data['PODIAG10'].value_counts(normalize=False, dropna=False)

In [ ]:
#Define the CPT to codes to exclude (anterior cervical procedures, thoracic and/or lumbar fusion, revision, intraspinal lesion).

data.loc[data['CPT'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['CPT'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT1'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT1'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT2'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT2'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT3'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT3'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT4'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT4'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT5'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT5'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT6'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT6'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT7'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT7'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT8'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT8'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT9'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT9'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['CONCPT10'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['CONCPT10'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT1'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT1'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT2'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT2'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT3'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT3'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT4'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT4'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT5'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT5'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT6'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT6'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT7'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT7'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT8'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT8'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT9'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT9'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['OTHERCPT10'] == 63075, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22554, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22585, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22551, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22552, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22220, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22556, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22558, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22610, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22612, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22630, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22632, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22633, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22634, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 20680, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22830, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22849, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22850, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22852, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 22855, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 63300, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 63301, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 63304, 'CPT_EX'] = 'Yes'
data.loc[data['OTHERCPT10'] == 63308, 'CPT_EX'] = 'Yes'

data.loc[data['CPT_EX'] != 'Yes', 'CPT_EX'] = 'No'

data['CPT_EX'].value_counts(dropna=False)

In [ ]:
#Exclude patients with CPT codes that were assigned to be excluding.

data = data[(data['CPT_EX'] == 'No')]

data['CPT_EX'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for elective surgeries.

data['ELECTSURG'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply inclusion criteria for elective surgeries.

data = data[(data['ELECTSURG'] == 'Yes')]

data['ELECTSURG'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for emergency surgery.

data['EMERGNCY'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply exclusion criteria for emergency surgery.

data = data[(data['EMERGNCY'] == 'No')]

data['EMERGNCY'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for anesthesia type.

data['ANESTHES'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply inclusion criteria for general anesthesia.

data = data[(data['ANESTHES'] == 'General')]

data['ANESTHES'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for surgical specialties.

data['SURGSPEC'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply inclusion criteria for surgical specialties.

data = data[(data['SURGSPEC'] == 'Neurosurgery') | (data['SURGSPEC'] == 'Orthopedics')]

data['SURGSPEC'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for wound class.

data['WNDCLAS'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply exclusion criteria for wound class.

data = data[(data['WNDCLAS'] == '1-Clean')]

data['WNDCLAS'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for preoperative sepsis.

data['PRSEPIS'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply exclusion criteria for preoperative sepsis.

data = data[(data['PRSEPIS'] == 'None')]

data['PRSEPIS'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for ASA Class.

data['ASACLAS'].value_counts(normalize=False, dropna=False)

In [ ]:
#Apply exclusion criteria for ASA class.

data = data[(data['ASACLAS'] != '4-Life Threat') & (data['ASACLAS'] != '5-Moribund') & (data['ASACLAS'] != 'None assigned')]

data['ASACLAS'].value_counts(normalize=False, dropna=False)

In [ ]:
#Create BMI column.

lbs_to_kg_ratio = 0.453592
inch_to_meter_ratio = 0.0254

data['HEIGHT'] *= inch_to_meter_ratio
data['WEIGHT'] *= lbs_to_kg_ratio

data['BMI'] = data['WEIGHT']/(data['HEIGHT']**2)
print(min(data['BMI']))
print(max(data['BMI']))

In [ ]:
#Check data for race.

data['RACE_NEW'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for ethnicity.

data['ETHNICITY_HISPANIC'].value_counts(normalize=False, dropna=False)

In [ ]:
#Simplify race and ethnicity columns.

data.loc[data['RACE_NEW'] == 'White', 'RACE'] = 'White'
data.loc[data['RACE_NEW'] == 'Black or African American', 'RACE'] = 'Black or African American'
data.loc[data['RACE_NEW'] == 'Asian', 'RACE'] = 'Asian'
data.loc[data['RACE_NEW'] == 'American Indian or Alaska Native', 'RACE'] = 'Other'
data.loc[data['RACE_NEW'] == 'Native Hawaiian or Other Pacific Islander', 'RACE'] = 'Other'
data.loc[data['RACE_NEW'] == 'Native Hawaiian or Pacific Islander', 'RACE'] = 'Other'
data.loc[data['RACE_NEW'] == 'Some Other Race', 'RACE'] = 'Other'

data.loc[data['ETHNICITY_HISPANIC'] == 'Yes', 'RACE'] = 'Hispanic'

data['RACE'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for transfer status.

data['TRANST'].value_counts(normalize=False, dropna=False)

In [ ]:
#Simplify transfer status column.

data.loc[data['TRANST'] == 'Not transferred (admitted from home)', 'TRANST'] = 'Not transferred'
data.loc[data['TRANST'] == 'Nursing home - Chronic care - Intermediate care', 'TRANST'] = 'Transferred'
data.loc[data['TRANST'] == 'From acute care hospital inpatient', 'TRANST'] = 'Transferred'
data.loc[data['TRANST'] == 'Outside emergency department', 'TRANST'] = 'Transferred'
data.loc[data['TRANST'] == 'Transfer from other', 'TRANST'] = 'Transferred'

data['TRANST'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for dyspnea.

data['DYSPNEA'].value_counts(normalize=False, dropna=False)

In [ ]:
#Simplify dyspnea column.

data.loc[data['DYSPNEA'] == 'No', 'DYSPNEA'] = 'No'
data.loc[data['DYSPNEA'] == 'MODERATE EXERTION', 'DYSPNEA'] = 'Yes'
data.loc[data['DYSPNEA'] == 'AT REST', 'DYSPNEA'] = 'Yes'

data['DYSPNEA'].value_counts(normalize=False, dropna=False)

In [ ]:
#Check data for diabetes status.

data['DIABETES'].value_counts(normalize=False, dropna=False)

In [ ]:
#Simplify diabetes column.

data.loc[data['DIABETES'] == 'NO', 'DIABETES'] = 'No'
data.loc[data['DIABETES'] == 'NON-INSULIN', 'DIABETES'] = 'Yes'
data.loc[data['DIABETES'] == 'INSULIN', 'DIABETES'] = 'Yes'

data['DIABETES'].value_counts(normalize=False, dropna=False)

In [ ]:
#Cast ASA class as ordered categorical.

cat_type1 = CategoricalDtype(categories=['1-No Disturb','2-Mild Disturb','3-Severe Disturb'], ordered=True)
data['ASACLAS'].astype(cat_type1)

In [ ]:
#Cast functional status as ordered categorical.

cat_type2 = CategoricalDtype(categories=['Unknown','Independent','Partiallly Dependent', 'Totally Dependent'], ordered=True)
data['FNSTATUS2'].astype(cat_type2)

In [ ]:
#Convert 90+ to 91 and AGE column to integer.

data.loc[data['AGE'] == '90+', 'AGE'] = 91
data['AGE'] = pd.to_numeric(data['AGE'], downcast='integer')

In [ ]:
#Show patients for each CPT code.

data['CPT'].value_counts()

In [ ]:
#Classify tumors into extradural vs. intradural and create a column named 'IEDUR' for it.

data.loc[data['CPT'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['CONCPT1'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['CONCPT2'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['CONCPT3'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['CONCPT4'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['CONCPT5'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['CONCPT6'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['CONCPT7'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['CONCPT9'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['CONCPT10'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['OTHERCPT1'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['OTHERCPT2'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['OTHERCPT3'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['OTHERCPT4'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['OTHERCPT5'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['OTHERCPT6'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['OTHERCPT7'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['OTHERCPT9'] == 22614, 'LEVELS'] = 'Multi'
data.loc[data['OTHERCPT10'] == 22614, 'LEVELS'] = 'Multi'

data.loc[data['LEVELS'] != 'Multi', 'LEVELS'] = 'Single'

data['LEVELS'].value_counts(dropna=False)

In [ ]:
#Define major complications.

data['MAJRCOMP'] = data['NWNDINFD'] + data['NORGSPCSSI'] + data['NDEHIS'] + data['NREINTUB'] + data['NPULEMBOL'] + data['NFAILWEAN'] + data['NRENAINSF'] + data['NOPRENAFL'] + data['NCNSCVA'] + data['NCDARREST'] + data['NCDMI'] + data['NOTHBLEED'] + data['NOTHDVT'] + data['NOTHSYSEP'] + data['NOTHSESHOCK']

In [ ]:
#Show number of major complications per patient.

data['MAJRCOMP'].value_counts()

In [ ]:
#Convert major complications into categorical data in a column named 'COMP'.

data.loc[data['MAJRCOMP'] == 0, 'COMP'] = 'No'
data.loc[data['MAJRCOMP'] >= 1, 'COMP'] = 'Yes'

In [ ]:
#Show major complications as categorical.

data['COMP'].value_counts()

In [ ]:
#See LOS per patient.

data['TOTHLOS'].value_counts()

In [ ]:
#See 75th percentile of LOS.
data.TOTHLOS.quantile(0.75)

In [ ]:
#Convert total length of stay into categorical data in a column named 'LOS'.

data.loc[data['TOTHLOS'] <= data.TOTHLOS.quantile(0.75), 'LOS'] = 'No'
data.loc[data['TOTHLOS'] > data.TOTHLOS.quantile(0.75), 'LOS'] = 'Yes'

In [ ]:
#Show major complications as categorical.

data['LOS'].value_counts(dropna=False)

In [ ]:
#Drop patients with unknown LOS.

data = data[data['LOS'].notna()]

In [ ]:
#Show readmission status.

data['READMISSION1'].value_counts(dropna=False)

In [ ]:
#Drop patients with unknown readmission status.

data = data[data['READMISSION1'].notna()]

In [ ]:
#Show readmission status after dropping patients with unknown readmission status.

data['READMISSION1'].value_counts(dropna=False)

In [ ]:
#Show discharge status.

data['DISCHDEST'].value_counts(dropna=False)

In [ ]:
#Convert discharge destination into binary data (home vs. non-home discharge) in a column named 'DISCHARGE'.

data.loc[data['DISCHDEST'] == 'Home', 'DISCHARGE'] = 'No'
data.loc[data['DISCHDEST'] == 'Facility Which was Home', 'DISCHARGE'] = 'No'
data.loc[data['DISCHDEST'] == 'Rehab', 'DISCHARGE'] = 'Yes'
data.loc[data['DISCHDEST'] == 'Skilled Care, Not Home', 'DISCHARGE'] = 'Yes'
data.loc[data['DISCHDEST'] == 'Separate Acute Care', 'DISCHARGE'] = 'Yes'
data.loc[data['DISCHDEST'] == 'Multi-level Senior Community', 'DISCHARGE'] = 'Yes'

In [ ]:
#Show discharge destination status after converting it to binary data.

data['DISCHARGE'].value_counts(dropna=False)

In [ ]:
#Drop patients with unknown discharge status.

data = data[data['DISCHARGE'].notna()]

In [ ]:
#Show discharge status after dropping patients with unknown discharge status.

data['DISCHARGE'].value_counts(dropna=False)

In [ ]:
#Check data.

data.shape

In [ ]:
#See all columns.

print(list(data.columns))

In [ ]:
#Drop unwanted columns.

drop = ['RACE_NEW', 'ETHNICITY_HISPANIC', 'CPT',  'DISCHDEST', 'ANESTHES', 'ELECTSURG', 'PRSEPIS', 'OTHERCPT1', 'OTHERCPT2', 'OTHERCPT3', 'OTHERCPT4', 'OTHERCPT5', 'OTHERCPT6', 'OTHERCPT7', 'OTHERCPT8', 'OTHERCPT9', 'OTHERCPT10', 'CONCPT1', 'CONCPT2', 'CONCPT3', 'CONCPT4', 'CONCPT5', 'CONCPT6', 'CONCPT7', 'CONCPT8', 'CONCPT9', 'CONCPT10', 'EMERGNCY', 'WNDCLAS', 'OPTIME', 'TOTHLOS', 'HTOODAY', 'NSUPINFEC', 'NWNDINFD', 'NORGSPCSSI', 'NDEHIS', 'NOUPNEUMO', 'NREINTUB', 'NPULEMBOL', 'NFAILWEAN', 'NRENAINSF', 'NOPRENAFL', 'NURNINFEC', 'NCNSCVA', 'NCDARREST', 'NCDMI', 'NOTHBLEED', 'NOTHDVT', 'NOTHSYSEP', 'NOTHSESHOCK', 'PODIAG', 'PODIAG10', 'STILLINHOSP', 'CPT_EX', 'MAJRCOMP']
data.drop(drop, axis=1, inplace=True)

In [ ]:
#Save data.

data.to_csv('/content/drive/MyDrive/NSQIP-PCF/pcf_clean.csv')

In [ ]:
#See categorical and continuous variables.

print('Numerical columns: {}'.format(list(data.select_dtypes('number').columns)))
print()
print('Categorical columns: {}'.format(list(data.select_dtypes('object').columns)))

In [ ]:
#Define numerical and categorical columns.

num_cols = ['AGE', 'HEIGHT', 'WEIGHT', 'PRSODM', 'PRBUN', 'PRCREAT', 'PRALBUM', 'PRBILI', 'PRSGOT', 'PRALKPH', 'PRWBC', 'PRHCT', 'PRPLATE', 'PRPTT', 'PRINR', 'PRPT', 'BMI']

cat_cols = ['SEX', 'INOUT', 'TRANST', 'SURGSPEC', 'DIABETES', 'SMOKE', 'DYSPNEA', 'FNSTATUS2', 'VENTILAT', 'HXCOPD', 'ASCITES', 'HXCHF', 'HYPERMED', 'RENAFAIL', 'DIALYSIS', 'DISCANCR', 'WNDINF', 'STEROID', 'WTLOSS', 'BLEEDDIS', 'TRANSFUS', 'ASACLAS', 'READMISSION1', 'RACE', 'LEVELS', 'COMP', 'LOS', 'DISCHARGE']

In [ ]:
#Check missing values for numerical columns.

missing_num = data[num_cols].isnull().mean().round(4).mul(100).sort_values(ascending=False)

missing_num = pd.DataFrame(missing_num)

missing_num.columns = ['Value']

missing_num = missing_num[missing_num['Value'] > 0]

print(missing_num.index)

missing_num = missing_num[missing_num['Value'] > 25]

missing_num = list(missing_num.index)

print(missing_num)

In [ ]:
#Drop numerical columns with missing values over 25%.

data.drop(missing_num, axis=1, inplace=True)

In [ ]:
#Define new numerical columns.

num_cols = [x for x in num_cols if x not in missing_num]
print(num_cols)

In [ ]:
#Impute missing numerical values.

num_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
data[num_cols] = num_imputer.fit_transform(data[num_cols])

In [ ]:
#Check numerical variables with missing values after imputation.

data[num_cols].isnull().mean().round(4).mul(100).sort_values(ascending=False)

In [ ]:
#Check missing values for categorical columns.

missing_cat = data[cat_cols].isnull().mean().round(4).mul(100).sort_values(ascending=False)

missing_cat = pd.DataFrame(missing_cat)

missing_cat.columns = ['Value']

missing_cat = missing_cat[missing_cat['Value'] > 0]

print(missing_cat.index)

missing_cat = missing_cat[missing_cat['Value'] > 25]

missing_cat = list(missing_cat.index)

print(missing_cat)

In [ ]:
#Drop categorical columns with missing values over 25%.

data.drop(missing_cat, axis=1, inplace=True)

In [ ]:
#Define new categorical columns.

cat_cols = [x for x in cat_cols if x not in missing_cat]
print(cat_cols)

In [ ]:
#Replace missing categorical values with 'Unknown'.

for col in cat_cols:
    data[col].fillna(value='Unknown', inplace=True)

In [ ]:
#Check missing values after imputation.

data[cat_cols].isnull().mean().round(4).mul(100).sort_values(ascending=False)

In [ ]:
#Save imputed data.

data.to_csv('/content/drive/MyDrive/NSQIP-PCF/pcf_imputed.csv')

In [ ]:
#RobustScale data.

data[num_cols] = RobustScaler().fit_transform(data[num_cols])

In [ ]:
#Normalize data.

data[num_cols] = MinMaxScaler().fit_transform(data[num_cols])

In [ ]:
#Save scaled data.

data.to_csv('/content/drive/MyDrive/NSQIP-PCF/pcf_scaled.csv')

In [ ]:
#One hot encoding for categorical values.

data_final = pd.get_dummies(data, columns = cat_cols, drop_first = True)

In [ ]:
#Save final data.

data_final.to_csv('/content/drive/MyDrive/NSQIP-PCF/pcf_final.csv')